# Diving into French Presidential Discourses

Research project for the course [Diving into Digital Public Spaces](https://syllabus.sciencespo.fr/cours/202120/195617.html) (Sciences Po, CEU) by Giulia Annaliese Paxton, Ryan Hachem, and Simone Maria Parazzoli. The course was taught by [Jean-Philippe Cointet](https://medialab.sciencespo.fr/equipe/jean-philippe-cointet/) (médialab, Sciences Po) and [Márton Karsai](https://networkdatascience.ceu.edu/people/marton-karsai) (Department of Network and Data Science, Central European University).  

Authors: Giulia Annaliese Paxton, Ryan Hachem, and Simone Maria Parazzoli  
Date: 07/04/2022

In [ ]:
!pip install minet
!pip install pdfminer.six
!pip install unidecode
!python -m spacy download fr_core_news_sm

In [2]:
import pandas as pd
from pdfminer.high_level import extract_text
from gensim.test.utils import datapath
from gensim import utils
import spacy
from collections import Counter
import unidecode
import gensim.models

## Emmanuel Macron (EM)
Steps:
1. Retrieve Twitter data with [minet](https://github.com/medialab/minet)
2. Retrieve textual data from EM's manifesto
3. Merge the data and prepare the corpus with the help of [spaCy](https://spacy.io)
4. Train a word embeddings model on the EM's corpus with [Gensim](https://radimrehurek.com/gensim/)
5. Find the most frequent words in the corpus
6. Analyse words shared with other candidates in EM's model

### 1. Retrieve Twitter data with minet

In [3]:
## create a .csv of EM's Twitter activity since 27 Jan 2022
!minet tw scrape tweets "(from:EmmanuelMacron) until:2022-03-27 since:2022-01-27" > tweets_EM.csv

## convert the .csv file in a data frame using pandas
df_tw_EM = pd.read_csv("./tweets_EM.csv")

## create a list of tweets selecting the 'text' column of the data frame
list_tw_EM = df_tw_EM['text'].values.tolist()
len(list_tw_EM)

Searching for "(from:EmmanuelMacron) until:2022-03-27 since:2022-01-27"


214

In [4]:
## EM affiliates' twitter activity

!minet tw scrape tweets "(from:RolandLescure OR from:ilanacicurelrem OR from:ebothorel OR from:mguevenoux OR from:StanGuerini OR from:JulienBargeton OR from:Ambroise_Mejean OR from:RichardFerrand OR from:MaudBregeon OR from:LauStmartin OR from:cedric_o OR from:JeanCASTEX OR from:franckriester OR from:BrunoLeMaire OR from:AgnesRunacher) until:2022-03-27 since:2022-01-27" > tw_EM_aff_all.csv

Searching for "(from:RolandLescure OR from:ilanacicurelrem OR from:ebothorel OR from:mguevenoux OR from:StanGuerini OR from:JulienBargeton OR from:Ambroise_Mejean OR from:RichardFerrand OR from:MaudBregeon OR from:LauStmartin OR from:cedric_o OR from:JeanCASTEX OR from:franckriester OR from:BrunoLeMaire OR from:AgnesRunacher) until:2022-03-27 since:2022-01-27"


In [5]:
## convert .csv affiliates' tweets in a list
df_tw_EM_aff_all = pd.read_csv("tw_EM_aff_all.csv")
list_tw_EM_aff_all = df_tw_EM_aff_all['text'].values.tolist()
print(list_tw_EM_aff_all[0])
print(len(list_tw_EM_aff_all))

@apollinederoche @CityOfBoston @CambMA @EnMarcheBoston @jeunesmacronFDE @EmmanuelMacron @EnMarcheUSA Bravo !!!
2522


In [6]:
## merge EM's and EM's affiliates lists
list_tw_EM_all = list_tw_EM + list_tw_EM_aff_all

### 2. Retrieve textual data from EM's manifesto

In [7]:
## i retrieve a string from the pdf of EM's manifesto using extract_text of the pdfminer package
## the cleaning process is specific for this manifesto and it depends on the output of extract_text
manif_EM = extract_text('/Users/simonemariaparazzoli/Documents/Università/Sciences Po/Diving into public digital spaces/research/manifesto_macron.pdf')
manif_clean_EM = manif_EM.replace('-\n','')
#manif_clean_EM = manif_clean_EM.replace('\n\n','---')
manif_clean_EM = manif_clean_EM.replace('\n','')
manif_clean_EM = manif_clean_EM.replace('\xa0',' ')
manif_clean_EM = manif_clean_EM.replace('\x0c',' ')
manif_clean_EM = manif_clean_EM.replace('.','---')
#print(repr(manif_clean_EM))

In [8]:
## convert the string of the manifesto into a list
list_manif_EM = manif_clean_EM.split("---")
list_manif_EM = [s for s in list_manif_EM if len(s)>20]
len(list_manif_EM)
#print(list_manif_EM)

429

### 3. Merge the data and prepare the corpus

In [18]:
## merge the two lists of tweets and of the manifesto 
list_EM = list_tw_EM_all + list_tw_EM_all + list_manif_EM
len(list_EM)

5901

In [19]:
## load a spacy model to retrieve stop words
nlp = spacy.load("fr_core_news_sm")
stop_words_fr = nlp.Defaults.stop_words
#new_sw = ["avec","la","les","le","pour","un","une","nous","ete","et","je"]
#stop_words_fr.add(new_sw)

## clean the list of tweets and manifesto to get rid of useless words and make the list content homogeneous
list_EM_clean = []
for i in list_EM:
    doc = nlp(i)
    tokens = [unidecode.unidecode(token.text).lower()for token in doc 
              if ( token.text not in stop_words_fr and
                  len(token.text)>1 and
                  token.like_url == False )]
    tokens_joined = ' '.join(tokens)
    list_EM_clean.append(tokens_joined)

In [20]:
## test the output of the cleaning process
print(list_EM[2401])
print("---")
print(list_EM_clean[2401])

@ppillaudvivien @emma_ducros « Mais je pense que la réalité se situe quelque part entre les deux… » = admettre qu’on s’est planté sans admettre qu’on s’est planté 😘

(Et sinon, je suis preneuse des sources bien entendu !)
---
@ppillaudvivien @emma_ducros mais realite situe part admettre plante admettre plante 

 et preneuse sources bien entendu


### 4. Train a word embeddings model on EM's corpus

In [21]:
## prepare the corpus as a class
class MyCorpus_EM:

    def __iter__(self):
        for i in list_EM_clean:
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(i,min_len=3)

In [51]:
## train the word embeddings model_EM
sentences = MyCorpus_EM()
model_EM = gensim.models.Word2Vec(sentences=sentences, min_count=10, vector_size=300, epochs=100)

In [52]:
## transform the corpus list (that is made of tweets and sentences from the manifesto)
## in a list containing all the words of the corpus as elements of the list
words_EM = []

for i in list_EM_clean:
    i_split = i.split(' ') #transform the i document into a list (split at blank space)
    words_EM.extend(i_split)

## clean the list of tokens
words_EM_clean = [x for x in words_EM 
                   if x not in stop_words_fr
                   if x != "\n\n"
                   if len(x)>1]

### 5. Find the most frequent words in EM's corpus

In [53]:
## find the 30 most common words using Counter
words_freq_EM = Counter(words_EM_clean)
common_words_EM = words_freq_EM.most_common(100)
print(common_words_EM)

[('france', 719), ('@emmanuelmacron', 579), ('francais', 490), ('ans', 445), ('faire', 399), ('ukraine', 380), ('avecvous', 362), ('president', 336), ("aujourd'hui", 318), ('europe', 316), ('pays', 284), ('pfue2022', 262), ('ete', 253), ('face', 249), ('contre', 246), ('soutien', 244), ('entreprises', 239), ('etat', 235), ('travail', 229), ('macron', 225), ('guerre', 224), ('demain', 221), ('numerique', 220), ('cotes', 219), ('matin', 214), ('emploi', 211), ('soir', 208), ('projet', 205), ('pouvoir', 195), ('engagement', 190), ('emmanuel', 187), ('the', 186), ('crise', 184), ('politique', 181), ('sommes', 180), ('@avecvous', 180), ('bravo', 176), ('ici', 176), ('2017', 171), ('campagne', 168), ('rien', 167), ('bien', 167), ('europeenne', 160), ('2021', 158), ('echanges', 154), ('directan', 150), ('emplois', 148), ('jamais', 144), ('to', 144), ('industrie', 144), ('5ansdeplus', 144), ('nouvelle', 141), ('prix', 140), ('monde', 138), ('-vous', 138), ('jeunes', 138), ('presidentielle', 13

### 6. Analyse words shared with other candidates in EM's model

In [54]:
## first attempt with the most_similar function on our corpus using our model_EM
result = model_EM.wv.most_similar(positive=['societe'], topn=30)
print(result)

[('civile', 0.39250850677490234), ('regulation', 0.30056849122047424), ('chantiers', 0.2841099798679352), ('securiser', 0.27117881178855896), ('terrain', 0.25665637850761414), ('unis', 0.25176456570625305), ('fortes', 0.2450779229402542), ('tensions', 0.24190083146095276), ('independante', 0.2415953278541565), ('acte', 0.24013370275497437), ('livre', 0.23982328176498413), ('hier', 0.23747079074382782), ('invitation', 0.23738056421279907), ('axes', 0.2352234423160553), ('omc_fr', 0.2341994047164917), ('benevoles', 0.2313159853219986), ('remarquable', 0.22785989940166473), ('ville', 0.22727517783641815), ('presenter', 0.22646686434745789), ('tiers', 0.2251552790403366), ('vivre', 0.22415730357170105), ('batir', 0.22258202731609344), ('inclusive', 0.21625125408172607), ('pleyben', 0.21530233323574066), ('passionnants', 0.21463359892368317), ('durable', 0.21414896845817566), ('vive', 0.20851033926010132), ('nouvelle', 0.20783545076847076), ('collectif', 0.20664462447166443), ('fierte', 0.2

In [55]:
print(len(list_tw_EM_all),',',len(list_EM_clean))

2736 , 5901


In [56]:
result = model_EM.wv.most_similar(positive=['france'], topn=20)
print(result)

[('emplois', 0.2867138683795929), ('amitie', 0.25499919056892395), ('allemagne', 0.2380618155002594), ('investissements', 0.23741856217384338), ('souveraine', 0.23265232145786285), ('leaders', 0.2318609356880188), ('cher', 0.22534003853797913), ('reformes', 0.22212257981300354), ('developpe', 0.21784482896327972), ('champions', 0.21756237745285034), ('stabilite', 0.2145223617553711), ('batteries', 0.2101590633392334), ('industriels', 0.21005551517009735), ('haut', 0.20936492085456848), ('creent', 0.20894888043403625), ('industrie', 0.20833072066307068), ('industriel', 0.20759893953800201), ('technologique', 0.20676590502262115), ('sudradio', 0.20539474487304688), ('convictions', 0.20199592411518097)]


In [57]:
result = model_EM.wv.most_similar(positive=['etat'], topn=20)
print(result)

[('stade', 0.2715955078601837), ('departement', 0.2645811140537262), ('edf', 0.26156777143478394), ('collectivites', 0.25369027256965637), ('milliards', 0.23938755691051483), ('majeur', 0.23447830975055695), ('meur', 0.23195526003837585), ('condamne', 0.2256711721420288), ('millions', 0.22518302500247955), ('public', 0.22259515523910522), ('moderniser', 0.22026847302913666), ('publics', 0.21899643540382385), ('besoins', 0.2187228798866272), ('secretaire', 0.21198654174804688), ('euros', 0.2118666172027588), ('investissement', 0.20987755060195923), ('mobilise', 0.20803701877593994), ('essentiel', 0.2079942673444748), ('mdeur', 0.20613782107830048), ('eviter', 0.20052911341190338)]


In [58]:
result = model_EM.wv.most_similar(positive=['souverainete'], topn=20)
print(result)

[('independance', 0.3575151264667511), ('transition', 0.35659801959991455), ('regulation', 0.32897332310676575), ('technologie', 0.32385340332984924), ('autonomie', 0.32258933782577515), ('alimentaire', 0.31640076637268066), ('revolution', 0.2978116571903229), ('puissance', 0.29708239436149597), ('union', 0.28364261984825134), ('stabilite', 0.28122997283935547), ('resilience', 0.2774677574634552), ('dunkerque', 0.2747935652732849), ('geants', 0.27333441376686096), ('majeur', 0.2621993124485016), ('technologique', 0.26218414306640625), ('filiere', 0.2611853778362274), ('menace', 0.25900769233703613), ('developper', 0.2584499716758728), ('encourager', 0.2573521137237549), ('strategie', 0.25693753361701965)]


In [59]:
result = model_EM.wv.most_similar(positive=['president'], topn=20)
print(result)

[('zelenskyyua', 0.34458622336387634), ('adresse', 0.29062095284461975), ('coeur', 0.2672080993652344), ('represente', 0.2569952607154846), ('moments', 0.25663095712661743), ('tomber', 0.2541256844997406), ('belfort', 0.2476288080215454), ('combattre', 0.24761347472667694), ('constitution', 0.24204552173614502), ('lecture', 0.24174457788467407), ('parole', 0.23796865344047546), ('zelensky', 0.2318795919418335), ('ukraine', 0.23063190281391144), ('agit', 0.22738219797611237), ('hautsdeseine', 0.22607778012752533), ('rassemblement', 0.22567763924598694), ('dictateur', 0.22309793531894684), ('lettre', 0.22107185423374176), ('presidente', 0.2193031907081604), ('crois', 0.21774788200855255)]


In [60]:
result = model_EM.wv.most_similar(positive=['politique'], topn=20)
print(result)

[('marsegalite', 0.26811984181404114), ('parite', 0.2674102485179901), ('sur', 0.26029253005981445), ('exportatrices', 0.2577246427536011), ('strategie', 0.24993257224559784), ('social', 0.24892987310886383), ('levier', 0.24553582072257996), ('attractivite', 0.22333374619483948), ('immense', 0.21505866944789886), ('carte', 0.21336114406585693), ('tout', 0.21138085424900055), ('ambitieuse', 0.20935970544815063), ('front', 0.20926091074943542), ('technologie', 0.20921362936496735), ('essentielle', 0.20783929526805878), ('resultat', 0.20532485842704773), ('reformes', 0.2050267904996872), ('fruits', 0.20266322791576385), ('portent', 0.20125819742679596), ('cree', 0.1991584450006485)]


In [61]:
result = model_EM.wv.most_similar(positive=['droit'], topn=20)
print(result)

[('oubli', 0.6005791425704956), ('malades', 0.4425801932811737), ('bloc', 0.3929089307785034), ('bielorusse', 0.3677496314048767), ('assurance', 0.36418187618255615), ('souverain', 0.3303214907646179), ('duree', 0.3274277448654175), ('populations', 0.3267963230609894), ('hemicycle', 0.3212445080280304), ('resolution', 0.3162866234779358), ('anciens', 0.3141126334667206), ('renforcons', 0.30837857723236084), ('deputes', 0.2944926917552948), ('pourtant', 0.29369187355041504), ('recours', 0.29347482323646545), ('cessez', 0.28630590438842773), ('avancee', 0.28331276774406433), ('hebergement', 0.2831461727619171), ('parce', 0.2807115912437439), ('garantir', 0.27973002195358276)]


In [62]:
result = model_EM.wv.most_similar(positive=['entreprise'], topn=20)
print(result)

[('medicaux', 0.3945430815219879), ('meur', 0.3786129355430603), ('bravo', 0.36840274930000305), ('salaries', 0.3613012433052063), ('specialisee', 0.3516063988208771), ('entrepreneurs', 0.3473721742630005), ('pme', 0.3358663320541382), ('fabrication', 0.3273876905441284), ('reconquete', 0.30171844363212585), ('accelere', 0.30084723234176636), ('start', 0.294516921043396), ('soutenus', 0.29021674394607544), ('euros', 0.2864997982978821), ('classes', 0.28354957699775696), ('classement', 0.2799932658672333), ('eti', 0.2795565724372864), ('recrute', 0.27618154883384705), ('plans', 0.27594107389450073), ('grandir', 0.2709389626979828), ('production', 0.27000802755355835)]


In [63]:
result = model_EM.wv.most_similar(positive=['economie'], topn=20)
print(result)

[('economiques', 0.27929869294166565), ('croissance', 0.2701939046382904), ('benefice', 0.26063910126686096), ('fruits', 0.2575156092643738), ('coherence', 0.2563030421733856), ('industrielle', 0.25540587306022644), ('opinion', 0.254231721162796), ('cree', 0.24920624494552612), ('crise', 0.2489301562309265), ('dynamisme', 0.24500364065170288), ('taux', 0.24333128333091736), ('publiques', 0.2423664778470993), ('excellence', 0.24028222262859344), ('electrique', 0.2398621290922165), ('inseefr', 0.2393648624420166), ('sauver', 0.23880535364151), ('accord', 0.23801885545253754), ('economique', 0.23745396733283997), ('rapidement', 0.23743559420108795), ('allemagne', 0.2351391613483429)]


In [64]:
result = model_EM.wv.most_similar(positive=['emploi'], topn=20)
print(result)

[('chomage', 0.3541828393936157), ('insertion', 0.3276729881763458), ('inseefr', 0.30996376276016235), ('industriel', 0.2987366020679474), ('developpement', 0.29236868023872375), ('ecologie', 0.29096877574920654), ('taux', 0.29053372144699097), ('net', 0.28942179679870605), ('reformes', 0.28503283858299255), ('prive', 0.26962611079216003), ('baisse', 0.26929333806037903), ('trimestre', 0.2652064859867096), ('emplois', 0.2588253319263458), ('rebond', 0.2581120729446411), ('deficit', 0.255900502204895), ('resultat', 0.24611583352088928), ('c_barbier', 0.24270416796207428), ('decarbonation', 0.24155834317207336), ('atteindre', 0.23606330156326294), ('route', 0.2326776683330536)]


In [65]:
result = model_EM.wv.most_similar(positive=['travail'], topn=20)
print(result)

[('devoirs', 0.264809787273407), ('classe', 0.2644297778606415), ('excellent', 0.25303220748901367), ('assurance', 0.25264453887939453), ('ppl', 0.25048938393592834), ('permettant', 0.2456965446472168), ('cree', 0.2432345151901245), ('unie', 0.2411782294511795), ('avancees', 0.22656971216201782), ('revenir', 0.2225690484046936), ('oubli', 0.22114194929599762), ('legislature', 0.21604299545288086), ('meilleurs', 0.21417823433876038), ('connait', 0.21358585357666016), ('commun', 0.2122085839509964), ('larem_an', 0.2121296525001526), ('missions', 0.21033771336078644), ('meilleures', 0.21031276881694794), ('comprendre', 0.20716224610805511), ('veux', 0.20542924106121063)]


In [66]:
result = model_EM.wv.most_similar(positive=['numerique'], topn=20)
print(result)

[('cybersecurite', 0.3605012893676758), ('geants', 0.3458486497402191), ('encadrer', 0.3429180681705475), ('regulation', 0.3018427789211273), ('conseillers', 0.294783353805542), ('scaleupeurope', 0.28806185722351074), ('revolution', 0.28752610087394714), ('innovation', 0.2872970998287201), ('deployer', 0.2852206826210022), ('mccourtinst', 0.2824726998806), ('potentiel', 0.2796661853790283), ('course', 0.2759546935558319), ('sciencespo', 0.2726476192474365), ('dsa', 0.27243414521217346), ('alimentation', 0.2716062664985657), ('numeriques', 0.2614743113517761), ('encourager', 0.2571374177932739), ('cloud', 0.2507874667644501), ('opendata', 0.2507260739803314), ('acteurs', 0.24773439764976501)]


In [67]:
result = model_EM.wv.most_similar(positive=['donnees'], topn=20)
print(result)

[('opendata', 0.46210750937461853), ('complete', 0.44924572110176086), ('milieu', 0.40065887570381165), ('versement', 0.3641936480998993), ('details', 0.35657253861427307), ('jeudi', 0.35465195775032043), ('tricolore', 0.34631577134132385), ('exigences', 0.33679986000061035), ('y_leduc', 0.32621875405311584), ('regles', 0.3213975131511688), ('futurs', 0.3196296989917755), ('humour', 0.3127623200416565), ('adapte', 0.3116649091243744), ('leve', 0.30962568521499634), ('jour', 0.3064456284046173), ('mix', 0.3059462010860443), ('agricole', 0.3055151402950287), ('ouverture', 0.3054846525192261), ('sobriete', 0.3008357286453247), ('demarches', 0.30048006772994995)]


In [68]:
result = model_EM.wv.most_similar(positive=['monde'], topn=20)
print(result)

[('excellence', 0.30965879559516907), ('entier', 0.3090592920780182), ('rayonner', 0.27269408106803894), ('alerte', 0.2588072419166565), ('take', 0.2254437506198883), ('soutenons', 0.2227087914943695), ('leaders', 0.2210460603237152), ('developpement', 0.2192710041999817), ('agricole', 0.21694041788578033), ('decarbonation', 0.21445736289024353), ('prochaines', 0.21174898743629456), ('crise', 0.20682430267333984), ('incroyable', 0.20521187782287598), ('veut', 0.20426273345947266), ('french', 0.20067793130874634), ('sauver', 0.19972366094589233), ('inde', 0.19939276576042175), ('villes', 0.1991419494152069), ('pere', 0.19664455950260162), ('clivages', 0.19465626776218414)]


In [69]:
result = model_EM.wv.most_similar(positive=['realite'], topn=20)
print(result)

[('candidate', 0.45125895738601685), ('reel', 0.40338411927223206), ('aucun', 0.3757425546646118), ('ceux', 0.3681643307209015), ('gauche', 0.3671541213989258), ('concretement', 0.36373183131217957), ('difference', 0.3368373513221741), ('salomesaque', 0.3336264193058014), ('soient', 0.33005350828170776), ('toujours', 0.3198540210723877), ('defakator_off', 0.3172747790813446), ('valeriepecresse', 0.3134477436542511), ('commence', 0.31072789430618286), ('eciotti', 0.3080361783504486), ('ailleurs', 0.3073420822620392), ('voter', 0.29858896136283875), ('explique', 0.29774728417396545), ('twitterfrance', 0.2951316237449646), ('frais', 0.2910027503967285), ('depassement', 0.2877015173435211)]


In [70]:
result = model_EM.wv.most_similar(positive=['societe'], topn=20)
print(result)

[('civile', 0.39250850677490234), ('regulation', 0.30056849122047424), ('chantiers', 0.2841099798679352), ('securiser', 0.27117881178855896), ('terrain', 0.25665637850761414), ('unis', 0.25176456570625305), ('fortes', 0.2450779229402542), ('tensions', 0.24190083146095276), ('independante', 0.2415953278541565), ('acte', 0.24013370275497437), ('livre', 0.23982328176498413), ('hier', 0.23747079074382782), ('invitation', 0.23738056421279907), ('axes', 0.2352234423160553), ('omc_fr', 0.2341994047164917), ('benevoles', 0.2313159853219986), ('remarquable', 0.22785989940166473), ('ville', 0.22727517783641815), ('presenter', 0.22646686434745789), ('tiers', 0.2251552790403366)]


In [71]:
result = model_EM.wv.most_similar(positive=['pouvoir'], topn=20)
print(result)

[('petrole', 0.31483057141304016), ('preoccupations', 0.3008726239204407), ('fiscalite', 0.2919941246509552), ('continuons', 0.2878381609916687), ('autant', 0.2857145369052887), ('continuer', 0.26868730783462524), ('particulier', 0.2628958523273468), ('tele', 0.25739815831184387), ('revenu', 0.2552710175514221), ('principe', 0.24455201625823975), ('jusqu', 0.23902544379234314), ('baisses', 0.23824700713157654), ('essentiel', 0.23684059083461761), ('raison', 0.23598481714725494), ('puissent', 0.23568430542945862), ('propagande', 0.23486894369125366), ('continuerons', 0.23474186658859253), ('ferons', 0.23346275091171265), ('debut', 0.23080070316791534), ('immobilier', 0.2298208624124527)]


In [72]:
result = model_EM.wv.most_similar(positive=['avenir'], topn=20)
print(result)

[('grandes', 0.3177250921726227), ('startups', 0.3160872161388397), ('propres', 0.2940877676010132), ('renouveau', 0.28046542406082153), ('reacteurs', 0.2789638340473175), ('defis', 0.27863243222236633), ('industrielle', 0.2756565511226654), ('volonte', 0.25575071573257446), ('devons', 0.250943660736084), ('economiques', 0.24860763549804688), ('defi', 0.23974715173244476), ('coherence', 0.23435664176940918), ('engagee', 0.2317994385957718), ('alimentaire', 0.22885219752788544), ('industrielles', 0.2228970229625702), ('sortis', 0.21831294894218445), ('sources', 0.21385742723941803), ('participe', 0.21074914932250977), ('trois', 0.21053199470043182), ('electrique', 0.2096056342124939)]


In [73]:
result = model_EM.wv.most_similar(positive=['histoire'], topn=20)
print(result)

[('troubles', 0.4124077558517456), ('aime', 0.40128448605537415), ('oublions', 0.3914104998111725), ('antisemitisme', 0.35829514265060425), ('paralympiques', 0.3389182984828949), ('amis', 0.33430105447769165), ('pendant', 0.3302885890007019), ('republicaine', 0.3302115499973297), ('passion', 0.3269307613372803), ('pernaut', 0.3244267404079437), ('chef', 0.3230729401111603), ('incroyable', 0.32047179341316223), ('olympiques', 0.30869489908218384), ('decouvrir', 0.3050886392593384), ('parcours', 0.30362215638160706), ('constitution', 0.30224111676216125), ('propos', 0.2975296378135681), ('montreal', 0.2945951223373413), ('clement', 0.2939325273036957), ('ecouter', 0.2909419536590576)]


In [74]:
result = model_EM.wv.most_similar(positive=['contre'], topn=20)
print(result)

[('desinformation', 0.43774381279945374), ('google', 0.3010644018650055), ('cyber', 0.29966282844543457), ('impact', 0.28182435035705566), ('propagande', 0.2779935598373413), ('inacceptable', 0.26453205943107605), ('membres', 0.2562917172908783), ('sputnik', 0.2543478012084961), ('adopte', 0.2462182641029358), ('protection', 0.24479813873767853), ('objectifs', 0.23501284420490265), ('bons', 0.23278702795505524), ('racine', 0.23185135424137115), ('interdiction', 0.2316015064716339), ('climatique', 0.22882404923439026), ('reponse', 0.2266305387020111), ('lutte', 0.2245924025774002), ('feu', 0.22286024689674377), ('chaines', 0.22267667949199677), ('war', 0.2207089513540268)]


In [75]:
result = model_EM.wv.most_similar(positive=['faut'], topn=20)
print(result)

[('necessaire', 0.2901647090911865), ('arrive', 0.2771615982055664), ('equilibre', 0.2691044509410858), ('scenario', 0.25582942366600037), ('consommation', 0.2502232789993286), ('energie', 0.2499023675918579), ('renouvelables', 0.24384473264217377), ('proteger', 0.23525957763195038), ('repondrons', 0.23425722122192383), ('porte', 0.231162890791893), ('instant', 0.22970962524414062), ('pire', 0.22712865471839905), ('disent', 0.2267685830593109), ('tout', 0.22565528750419617), ('paix', 0.22305358946323395), ('aupres', 0.22277460992336273), ('agir', 0.22091606259346008), ('realite', 0.22073490917682648), ('appartient', 0.2116556167602539), ('debatjeuneslci', 0.21021220088005066)]


In [76]:
result = model_EM.wv.most_similar(positive=['crise'], topn=20)
print(result)

[('resistance', 0.28857606649398804), ('facture', 0.2815491855144501), ('exceptionnel', 0.2671382427215576), ('cap', 0.26608553528785706), ('protege', 0.26607027649879456), ('dette', 0.2586592733860016), ('sans', 0.2526896595954895), ('voisins', 0.24974706768989563), ('montee', 0.24923528730869293), ('economie', 0.24893012642860413), ('soutenu', 0.24203680455684662), ('impots', 0.23680707812309265), ('historique', 0.23249028623104095), ('sortis', 0.23093432188034058), ('consequences', 0.22869299352169037), ('mesures', 0.22660699486732483), ('permis', 0.22550080716609955), ('mobilise', 0.22538582980632782), ('accelere', 0.22480855882167816), ('chaines', 0.22244565188884735)]
